# Teorema de Bayes

Probabilidad condicional: Probabilidad que ocurra un evento dado que otro ocurrió.<img src="interseccion_conjuntos.png" alt="Drawing" style="width: 300px;"/>

La probabilidad condicional se denota como: $P(A|B)$ y se lee probabilidad del evento $A$ dado el evento $B$

Teorema de Bayes:
    $$P(A|B)=\frac{P(B|A)P(A)}{P(B)}$$

Expresa la probabilidad condicional $P(A|B)$, en términos de la probabilidad de la intersección de los eventos y de las probabilidades de cada evento por separado.

Ejemplo: Se que un test de coronavirus tiene un 90% sensitividad and 95% especificidad y me dio positivo, cual es la probabilidad que tenga coronavirus?
Usando Bayes: $$P(\text{Tengo Corona y me dio positivo}) = \frac{P(\text{Me de positivo y tenga corona})P(\text{Tenga Corona})}{P(\text{Me de positivo})}$$

Calculamos las probabilidades:
    $$P(\text{Me de positivo y tenga corona}) = 0.9\text{ (sensitividad del test)}$$
 

   $$P(\text{Tenga Corona}) = 0.05 (\text{  ej 5% de la población)}$$


$$ P(\text{Me de positivo}) = P(\text{Me de positivo|Tenga corona})*P(\text{Tenga corona}) + P(\text{Me de positivo|No tenga corona})*P(\text{No tenga corona})$$

$$P(\text{Me de positivo}) = 0.9*0.05+0.05*(1-0.05)=0.0925$$

$$P(\text{Tengo Corona|Me dio positivo})=\frac{0.9*0.05}{0.0925} = 0.4864$$


Ejercicio: calcular de los que dieron negativos, cuantos realmente no tenían.

Casos de uso: Estimación de parameter, testeo de hipótesis, cálculo de intervalos de confianza, cualquier tipo de inferencia bayesiana, etc.

## Divergencia de Kullback–Leibler o KL-Divergence

Se define como: $$D_{KL}=\sum_{x \in X} P(x)log( \frac{P(x)}{Q(x)})$$

- Da una noción de similaridad entre distribuciones de probabilidades

- No es una métrica ya que no cumple la desigualdad triangular, por lo tanto siempre conviene calcularla relativa a otra distribución

- Se relaciona con el campo de teoría de información: Dicha cantidad equivale a la información mutua de P respecto Q, es decir, cuanta información de la distribución de probabilidad P hay en la distribución de Q

- El inconveniente de este cálculo es que generalmente no se conocen las distribuciones de probabilidades de las variables y siempre esta la cuestión de: mi histograma se aproxima lo suficiente a mi distribución real?

## [Pymc3: Probabilistic Programming in Python](https://docs.pymc.io/)

- Tiene distribuciones de probabilidades predefinidas 
- Permite hacer cálculos de probabilidades posterior de a priori a partir de datos

Ejemplo: **Autoregresiones Bayesianas**

In [40]:

%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pymc3 as pm


sns.set(rc={'figure.figsize': (12,8)})

In [ ]:
sigma = 1.0

with pm.Model() as ar1:
    phi1 = pm.Normal('phi_1', mu=0, sigma=sigma)
    data = pm.AR('p', phi1, observed=df['Cierre'])
    trace = pm.sample(10000, tune=4000)
    map_ar1 = pm.find_MAP()

pm.traceplot(trace);